# Import section

In [1]:
# Data handling
import pandas as pd
import numpy as np

# Model evaluation and metrics
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score

# pyFUME and AUK
from pyfume import SugenoFISTester
from AUK import AUK

# For saving/loading data
import pickle
import os

# Data loading

In [2]:
# Load best hyperparameters from pkl file
best_hyperparameters = pd.read_pickle('model_training/best_hyperparameters.pkl')

# Extract the first row from the 'params' column
params_dict = best_hyperparameters['params'].iloc[0]

# Ensure the 'params' column contains a dictionary
if isinstance(params_dict, dict):
    # Dynamically create variables from the dictionary keys
    for key, value in params_dict.items():
        locals()[key] = value

    # Derive dataframe_name from the params_dict or directly from 'dataframe_name' key
    if 'dataframe_name' in params_dict:
        dataframe_name = params_dict['dataframe_name']

        # Replace 'train' with 'test' in the dataframe name
        dataframe_name = dataframe_name.replace('train', 'test', 1)

        # Construct the .pkl file path
        base_folder = 'feature_selection'
        pkl_file_path = f'{base_folder}/{dataframe_name}.pkl'

        # Load the test dataframe
        test_df = pd.read_pickle(pkl_file_path)

        # Print to verify
        print(f"Loaded dataframe from: {pkl_file_path}")
    else:
        print("Error: 'dataframe_name' not found in params_dict.")
else:
    print("Error: The 'params' column does not contain a dictionary.")

Loaded dataframe from: feature_selection/test_df_all_features_10.pkl


In [3]:
# drop extra columns that are not needed for the model
df_test_clean = test_df.drop(columns=['eid', 'p130894', 'p130895', 'p53_i0'])

# extract variable names
variable_names = df_test_clean.drop(columns=['target']).columns.tolist()

# Split the test data into features and ground truth
X_test = df_test_clean.drop(columns=['target'])
y_test = df_test_clean['target']

# Convert X_test and y_test to NumPy arrays
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

# Load the saved model
with open('model_training/final_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
    
# Create a tester object to evaluate the model on the test data
tester = SugenoFISTester(model=loaded_model, test_data=X_test, variable_names=variable_names, golden_standard=y_test)

# Predict probabilities for the test data
y_pred_proba = tester.predict()

# Extract the first element of the tuple (the predicted probabilities)
y_pred_proba = y_pred_proba[0]

In [4]:
# Convert probabilities to binary predictions using a threshold (e.g., 0.5)
y_pred = (y_pred_proba > 0.5).astype(int)

In [5]:
# Ensure the index of y_pred_series matches df_test
y_pred_series = pd.Series(y_pred, index=test_df.index, name='y_pred')
y_pred_proba_series = pd.Series(y_pred_proba, index=test_df.index, name='y_pred_proba')

# create copy of df_test

df_test_incl_predictions = test_df.copy()

# Insert the new columns after the 'target' column
df_test_incl_predictions.insert(df_test_incl_predictions.columns.get_loc('target') + 1, 'y_pred', y_pred_series)
df_test_incl_predictions.insert(df_test_incl_predictions.columns.get_loc('target') + 2, 'y_pred_proba', y_pred_proba_series)

In [6]:
df_test_incl_predictions.head()

,p728_i0_Category_B,p1200_i0_Category_E,p10895_i0_Category_E,p6222_i0,p4041_i0_Category_B,p6148_i0_Category_E,p3506_i0_Category_E,p1418_i0_Category_B,p4295_i0_Category_E,p1598_i0_Category_C,target,y_pred,y_pred_proba,eid,p130894,p130895,p53_i0
23,0.0,0.0,0.0,-0.120863,0.0,1.0,0.0,0.0,0.0,0.0,0,1,0.623272,ID_24,2012-03-28,Category_E,2009-02-16
67,0.0,0.0,0.0,0.986363,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.528376,ID_68,2010-06-18,Category_D,2009-05-01
81,0.0,0.0,0.0,0.959394,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.528376,ID_82,2019-01-11,Category_C,2007-05-09
590,0.0,0.0,0.0,0.235234,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0.504860,ID_591,2019-05-19,Category_E,2007-03-03
383,0.0,0.0,0.0,-0.377298,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.481396,ID_384,2014-09-11,Category_C,2007-04-21


# Calculate Metrics

In [7]:
# Calculate accuracy and AUC
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
print(accuracy)
print(auc)

0.6396694214876033
0.7409350518622259


In [8]:
# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix_df = pd.DataFrame(conf_matrix, 
                                   index=['Actual 0', 'Actual 1'], 
                                   columns=['Predicted 0', 'Predicted 1'])

In [9]:
confusion_matrix_df.head()

,Predicted 0,Predicted 1
Actual 0,214,99
Actual 1,119,173


In [10]:
# Calculate precision and recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Precision: {precision}')
print(f'Recall: {recall}')

Precision: 0.6360294117647058
Recall: 0.5924657534246576


In [11]:
# calculate AUK
auk_class = AUK(probabilities=y_pred_proba, labels=y_test)
        
# Calculate AUK
auk = auk_class.calculate_auk()
print(auk)

0.23940794100573437


In [12]:
# replace columns names 

# Dictionary for mapping old column names to new names
column_mapping = {
    'p2090_i0_Yes': 'Seen doctor (GP) for nerves, anxiety, tension or depression | Yes',
    'p2090_i0_No': 'Seen doctor (GP) for nerves, anxiety, tension or depression | No',
    'p2100_i0_No': 'Seen a psychiatrist for nerves, anxiety, tension or depression | No',
    'p1920_i0_Yes': 'Mood swings | Yes',
    'p1960_i0_No': 'Fed-up feelings | No',
    'p1920_i0_No': 'Mood swings | No',
    'p137_i0': 'Number of treatments/medications taken',
    'p2100_i0_Yes': 'Seen a psychiatrist for nerves, anxiety, tension or depression | Yes',
    'p2050_i0_Not_at_all': 'Frequency of depressed mood in last 2 weeks | Not at all',
    'p1980_i0_Yes': 'Worrier / anxious feelings | Yes'
}

# Rename columns in the DataFrame using the mapping
df_test_incl_predictions.rename(columns=column_mapping, inplace=True)

In [13]:
# check false positives
# Filter df_test_incl_predictions for rows where y_pred is 1 and target is 0
false_positives = df_test_incl_predictions[(df_test_incl_predictions['y_pred'] == 1) & 
                                       (df_test_incl_predictions['target'] == 0)]

false_positives.head(5)

,p728_i0_Category_B,p1200_i0_Category_E,p10895_i0_Category_E,p6222_i0,p4041_i0_Category_B,p6148_i0_Category_E,p3506_i0_Category_E,p1418_i0_Category_B,p4295_i0_Category_E,p1598_i0_Category_C,target,y_pred,y_pred_proba,eid,p130894,p130895,p53_i0
23,0.0,0.0,0.0,-0.120863,0.0,1.0,0.0,0.0,0.0,0.0,0,1,0.623272,ID_24,2012-03-28,Category_E,2009-02-16
67,0.0,0.0,0.0,0.986363,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.528376,ID_68,2010-06-18,Category_D,2009-05-01
355,0.0,0.0,0.0,-0.896725,0.0,1.0,0.0,0.0,0.0,1.0,0,1,0.629303,ID_356,2019-08-31,Category_D,2010-09-04
196,0.0,0.0,0.0,1.563542,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0.597583,ID_197,2012-03-20,Category_C,2008-11-27
347,0.0,0.0,0.0,-2.216768,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.516054,ID_348,2016-12-13,Category_E,2009-12-13


In [14]:
# Filtered DataFrames for true positives and false positives
true_positives = df_test_incl_predictions[(df_test_incl_predictions['y_pred'] == 1) & (df_test_incl_predictions['target'] == 1)]
false_positives = df_test_incl_predictions[(df_test_incl_predictions['y_pred'] == 1) & (df_test_incl_predictions['target'] == 0)]

# Calculating mean and standard deviation for the first ten columns
mean_std_true_positives = true_positives.iloc[:, :10].agg(['mean', 'std'])
mean_std_false_positives = false_positives.iloc[:, :10].agg(['mean', 'std'])

In [15]:
mean_std_true_positives.head()

,p728_i0_Category_B,p1200_i0_Category_E,p10895_i0_Category_E,p6222_i0,p4041_i0_Category_B,p6148_i0_Category_E,p3506_i0_Category_E,p1418_i0_Category_B,p4295_i0_Category_E,p1598_i0_Category_C
mean,0.161850,0.0,0.0,0.215111,0.156069,0.300578,0.0,0.167630,0.0,0.306358
std,0.369382,0.0,0.0,1.000239,0.363975,0.459840,0.0,0.374622,0.0,0.462319


In [16]:
mean_std_false_positives.head()

,p728_i0_Category_B,p1200_i0_Category_E,p10895_i0_Category_E,p6222_i0,p4041_i0_Category_B,p6148_i0_Category_E,p3506_i0_Category_E,p1418_i0_Category_B,p4295_i0_Category_E,p1598_i0_Category_C
mean,0.0,0.0,0.0,-0.041901,0.0,0.444444,0.0,0.0,0.0,0.414141
std,0.0,0.0,0.0,1.197068,0.0,0.499433,0.0,0.0,0.0,0.495080


# export predictions and ground truth

In [17]:
# Define the folder path
folder_name = "model_evaluation"

# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'model_evaluation' created.


In [18]:
# export predictions
np.savez('model_evaluation/predictions.npz', y_pred=y_pred, y_pred_proba=y_pred_proba, y_test=y_test)